#                           What Makes a Movie Profitable? Analysis of Movie Returns and Associated Factors



## 1. Business Understanding 
### 1.1 Introduction

Microsoft Corporation is a multinational technology company that develops, manufactures, licenses, supports, and sells computer software, consumer electronics, and personal computers. It is known for its flagship products like the Windows operating system, Microsoft Office suite, and Azure cloud computing platform. Product diversification into new sectors enables a corporation reduce dependence on a single industry, tap into new markets, cross-sell, adapt to evolving consumer needs, and sustain long-term success. 

 Diversifying to create original video content can help Microsoft expand their brand presence and reach a wider audience, enhancing their overall visibility and reputation. Also, movies provide opportunities for cross-promotion and product placements, allowing Microsoft to showcase its products and services to a massive audience.   Venturing into movies can open up new revenue streams through licensing and distribution agreements. 


### 1.2  Business Problem
Diversification should be approached strategically and based on evidence from market research for successful expansion.  There is a need to understand market trends particularly levels of investment and returns. Also understanding of customer preference is indispensable   in order to invest in products which resonate with the demand and have significant returns. This understanding enables one to make informed decisions, tailor products to the target market, develop effective marketing strategies, and mitigate potential risks. By gaining insights into the new market's opportunities and challenges, a corporation can position themselves for successful diversification and increase the likelihood of achieving desired outcomes.

Companies have accrued significant profits from the movie industry. However, this is not guaranteed as the returns are determined by a number of risk factors.  In this regard, I analyze historical data on varied variables to determine what determine profits from movies to inform investment decisions.  


### 1.3 Main Objective

To determine factors that influence profitability of movie genres.

#### Specific Objectives
1. What are the top produced genres of movies?
2. Do average rating, runtimes, budget and production budget influence profitability of movies?
3. What genres should the company consider investing in?


### 1.4 Data Understanding 

The data used was downloaded from the Box Office Mojo and IMDB. Inline with the objective, the analysis focused on variables which are defined below. 

**Genres**: Category of artistic composition, characterized by similarities in form, style, or subject matter for a piece of content.

**Production budget**: The production cost of a movie.

**Worldwide gross**: Box office revenues from all countries the movie sold.

**Profit**:	Computed as worldwide gross less production budget

### 1.5 Document Organization
1. Loading and Checking Data
2. Data Cleaning
3. Exploratory Data Analysis
4. Conclusions and Recommendations


### 2 Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### 3 Reading Data 

In [ ]:
#Loads the data from three csv files which have columns  that align with the  main and specific objectives 
Movie_budget_revenues = pd.read_csv('tn.movie_budgets.csv')
Title_ratings = pd.read_csv('title.ratings.csv')
Movie_basics = pd.read_csv('title.basics.csv')

### 4 Data Wrangling
#### 4.1 Joining Datasets

The three csv files are joined to be one then irrelevant columns are dropped from the combined data set 

In [ ]:
# The title ratings and movie basic have a common column tconst which is  alphanumeric unique identifier of a movie title.
#tconst was used to  combined the datasets 
combined = pd.merge(Title_ratings, Movie_basics, on='tconst', how='left')

In [ ]:
# The combined above and Movie_budget_revenues dataset have a common column of movie title
# The column movie was used to combine the datasets 
# First have change the column name for left joining 
combined  = combined.rename(columns={'original_title': 'movie'})

# Using outer join to  merge combined datasets of  movie rating and movie basic with  budget and revenue 
final_data = pd.merge(Movie_budget_revenues, combined, on='movie', how='left')

#### 4.2 Dropping irrelevant columns

In [ ]:
#The loaded data from three files  has irrelevant columns in relation to the objective.

final_data= final_data.drop (['id','movie','release_date', 'tconst', 'numvotes', 'primary_title','start_year', 'domestic_gross'], axis=1)

##### The dataset has five columns namely; production_budget, worldwide_gross, averagerating, runtime_minutes and genres 

#### 4.3 Checking the Data

In [ ]:
#specifying  that all floating-point numbers should be displayed with 2 decimal places. 
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Determining the no. of records in  dataset
final_data.shape

The final dataset has 6297 rows  and  5 columns

In [ ]:
# Previewing the top of  dataset
final_data.head()

In [ ]:
# Previewing the bottom of  dataset
final_data.tail()

In [ ]:
# Checking whether each column has an appropriate datatype
final_data.dtypes

##### Genres, average rating and run time minutes are of the correct data type which is      object and float64 respectively. Production budget and worldwide gross are of object data type which should be corrected to float64

### 4.4 Data Cleaning

In [ ]:
#Remove $ and comma from production budget  and wordlwide gross then converting  to float 
final_data['production_budget'] = final_data['production_budget'].str.replace("$", "")
final_data['production_budget'] = final_data['production_budget'].str.replace(",", "")
final_data['production_budget'] = final_data['production_budget'] .astype(float)

In [ ]:
final_data['production_budget'].describe()

In [ ]:
#Remove $ and comma from production budget  and wordlwide gross then converting  to float 
final_data['worldwide_gross'] = final_data['worldwide_gross'].str.replace("$", "")
final_data['worldwide_gross'] = final_data['worldwide_gross'].str.replace(",", "")
final_data['worldwide_gross'] = final_data['worldwide_gross'] .astype(float)

In [ ]:
final_data['worldwide_gross'].describe()

In [ ]:
# Checking for missing values 
final_data.isna().sum()

 ##### Genres, average rating and run time minutes have missing values.

In [ ]:
#Dropping missing values. 
columns_to_drop_na = ['averagerating', 'runtime_minutes', 'genres']
final_data.dropna(subset=columns_to_drop_na, inplace=True)



##### Genres, average rating and run time minutes have missing values were dropped given back refill nor forward refill is applicable.

In [ ]:
#Checking if all missing values have been dropped
final_data.isna().sum()

In [ ]:
# Checking for duplicated rows and ispecting any if there are
duplicate_rows = final_data[final_data.duplicated()].sum()
print (duplicate_rows)

#### 4.5 Data Decisions 
I decided to focus only on productions budgets  and  a worldwide gross of more than  $10million.

In [ ]:
# final_data['production_budget'] = final_data['production_budget'] >= 100000000
# final_data['worldwide_gross'] = final_data['worldwide_gross'] >= 100000000


##### Decided to focus on movies with a rating of five and above

In [ ]:
final_data = final_data['averagerating'] >= 5

In [ ]:
#Compute profit from production budget and worldwide gross
final_data['Profit'] = final_data['worldwide_gross'] - final_data['production_budget']


## 5 Exploratory Data Analysis
### 5.1  Univariate Analysis

In [ ]:
# AGroup movies by genre and rank them
grouped_data = final_data.groupby('genres').size().sort_values(ascending=False).reset_index(name='count')
# Display the top 10 genres with adjusted index starting from 1
top_10_genres = grouped_data.head(10).reset_index(drop=True)
top_10_genres.index = top_10_genres.index + 1
top_10_genres[['genres', 'count']]


Drama is the most common genre with a count of 211. Documentary and Comedy follow with 108 and 88 counts, respectively. Comedy, adventure, animation, and romance also appear frequently.

In [ ]:
# Group movies by genre and calculate the average rating for top 10 genres 
grouped_genre_averagerating = final_data.groupby('genres')['averagerating'].mean().reset_index()

# Sort the genres based on the averagerating in descending order
sorted_genre_averagerating = grouped_genre_averagerating.sort_values(by='averagerating', ascending=False)

# Select the top 10 genres as per averagerating
top_10_genres_averagerating = sorted_genre_averagerating.head(10)

# Reset the index and adjust it to start from 1
top_10_genres_averagerating = top_10_genres_averagerating.reset_index(drop=True)
top_10_genres_averagerating.index = top_10_genres_averagerating.index + 1

# Display the top 10 genres with their average ratings
top_10_genres_averagerating[['genres', 'averagerating']]

In [ ]:
# Group movies by genre and calculate the total production budget for top 10 genres
grouped_genre_budget = final_data.groupby('genres')['production_budget'].sum().reset_index()

# Sort the genres based on the worldwide production budget in descending order
sorted_genre_budget = grouped_genre_budget.sort_values(by='production_budget', ascending=False)

# Select the top 10 genres as per production budget
top_10_genres_budget = sorted_genre_budget.head(10)

# Reset the index and adjust it to start from 1
top_10_genres_budget.index = range(1, len(top_10_genres_budget) + 1)

# Display the top 10 genres as per production budget
top_10_genres_budget[['genres', 'production_budget']]

Production budgets differ across genres. Action, adventure, and Sci-Fi movies have the highest production budget followed by adventure, animation, and comedy. Other genres like drama, documentary, and comedy also have significant production budgets.

In [ ]:
# Group movies by genre and calculate the total worldwide gross for top 10 genres
grouped_genre_worldwidegross = final_data.groupby('genres')['worldwide_gross'].sum().reset_index()

# Sort the genres based on the worldwide gross in descending order
sorted_genre_worldwidegross = grouped_genre_worldwidegross.sort_values(by='worldwide_gross', ascending=False)

# Select the top 10 genres as per worldwide_gross
top_10_genres_worldwidegross = sorted_genre_worldwidegross.head(10)

# Reset the index and adjust it to start from 1
top_10_genres_worldwidegross.index = range(1, len(top_10_genres_worldwidegross) + 1)

# Display the genres and their worldwide gross
top_10_genres_worldwidegross[['genres', 'worldwide_gross']]

Action, adventure, and Sci-Fi movies generated the highest worldwide gross followed by adventure, animation, and comedy. Genres like drama, comedy, and documentary also achieved substantial worldwide gross revenue.

In [ ]:
# Group movies by genre and calculate the total worldwide  profit for top 10 genres
grouped_genre_profit = final_data.groupby('genres')['Profit'].sum().reset_index()

# Sort the genres based on the profit in descending order
sorted_genre_profit = grouped_genre_profit.sort_values(by='Profit', ascending=False)

# Select the top 10 profitable genres
top_10_genres_profit = sorted_genre_profit.head(10)

# Reset the index and adjust it to start from 1
top_10_genres_profit.index = range(1, len(top_10_genres_profit) + 1)

# Display the genres and their profit
top_10_genres_profit[['genres', 'Profit']]

There is a similar pattern as with worldwide gross where action, adventure, and Sci-Fi movies generated the highest profit of followed by adventure, animation, and comedy. Similarly, drama, comedy, and documentary also accrued notable profitability.

## 5.2 Correlation

In [ ]:
#Correlation between profit and averagerating
correlation_profit_averagerating = final_data['Profit'].corr(final_data['averagerating'])
correlation_profit_averagerating = round(correlation_profit_averagerating, 2)
print("The correlation between profit and averagerating is", correlation_profit_averagerating )


In [ ]:
#Correlation between profit and production budget
correlation_profit_productionbudget = final_data['Profit'].corr(final_data['production_budget'])
correlation_profit_productionbudget = round(correlation_profit_productionbudget, 2)
print("The correlation between profit and production_budget is",correlation_profit_productionbudget)


In [ ]:
#Correlation between profit and  run time minutes
correlation_profit_runtime_minutes = final_data['Profit'].corr(final_data['runtime_minutes'])
correlation_profit_runtime_minutes  = round(correlation_profit_runtime_minutes , 2)
print("The correlation between profit and runtime_minutes is",correlation_profit_runtime_minutes)

### 5.3 Visualization 

In [ ]:
# Bar graph between for top 10 genres
plt.figure(figsize=(10, 6))
plt.barh (top_10_genres['genres'], top_10_genres['count'])
plt.xlabel('Genres')
plt.ylabel('Count')
plt.title('Top 10 Genres')


In [ ]:
# Bar graph between for top 10 genres and production budget 
plt.figure(figsize=(10, 6))
plt.barh (top_10_genres_budget['genres'], top_10_genres_budget['production_budget'])
plt.xlabel('Production budget in $')
plt.ylabel('Genres')
plt.title('Production Budget for Top 10 Genres')
plt.ticklabel_format(style='plain', axis='x')

In [ ]:
# Bar graph between for top 10 Genres and worldwide gross
plt.figure(figsize=(10, 6))
plt.barh(top_10_genres_worldwidegross['genres'], top_10_genres_worldwidegross['worldwide_gross'])
plt.xlabel('Worldwide Gross in $')
plt.ylabel('Genres')
plt.title('Worldwide Gross for Top 10 Genres')
plt.ticklabel_format(style='plain', axis='x')

In [ ]:
# Bar graph between for top 10 genres and profit
plt.figure(figsize=(10, 6))
plt.barh(top_10_genres_profit['genres'], top_10_genres_profit['Profit'])
plt.xlabel('Profit in $')
plt.ylabel('Genres')
plt.title('Top 10 Profitable Genres')
plt.ticklabel_format(style='plain', axis='x')

In [ ]:
# Bar graph between for top 10 genres and average rating
plt.figure(figsize=(10, 6))
plt.barh (top_10_genres_averagerating['genres'], top_10_genres_averagerating['averagerating'])
plt.xlabel('Averagerating')
plt.ylabel('Genres')
plt.title('Top 10 Rated Genres')
plt.ticklabel_format(style='plain', axis='x')

In [ ]:
# Create the scatter plot
plt.scatter(final_data['production_budget'], final_data['Profit'])
plt.title("Production Budget vs. Profit")
plt.xlabel("Production Budget")
plt.ylabel("Profit")
plt.ticklabel_format(useOffset=False, style='plain')

### 6 Conclusions

-  There is a positive correlation between profit and production budget (0.65) and a weaker positive correlation with average rating (0.19) and runtime minutes (0.20). These correlations suggest that higher production budgets tend to result in higher profits, while the impact of average rating and runtime is comparatively smaller.

- A comparison of top 10 genres and top-rated genres with gross and profitability shows high ratings do not guarantee movie profitability.

- Action, Adventure, Sci-Fi and Adventure, Animation, Comedy are the top genres with the highest production budgets and profits.

- Drama, Documentary, and Adventure, Drama, Sci-Fi receive high ratings with moderate production budgets.

- Drama, Comedy, and Adventure, Animation, Comedy are popular genres with moderate production budgets and modest returns 

- Investing in higher production budgets can potentially lead to higher profits. This indicates that allocating more financial resources to the production of a film could be a viable strategy for maximizing returns on investment. However there are other factors influencing profitability in the film industry. As such there is need for further research.


### 7 Recommendations 
* While choosing the movie to produce, genre, production budget and profitability should be the main factors followed by average rating
- Focus on high-budget genres with strong profitability: Action, Adventure, Sci-Fi, Adventure, Animation, Comedy, and Action, Adventure, Fantasy have both high production budgets and significant profits. Investing in these genres can offer potential returns on investment due to their popularity and financial success.

- Consider genres with high ratings and moderate production budgets: Genres like Drama, Documentary, and Adventure, Drama, Sci-Fi receive high ratings and have moderate production budgets. These genres indicate a balance between critical acclaim and financial viability, making them promising investment options.

- Explore opportunities in popular genres with moderate production budgets: Drama, Comedy, and Adventure, Animation, Comedy are popular genres with moderate production budgets. Investing in these genres can tap into widespread audience appeal while managing production costs effectively, potentially leading to profitability.
